In [1]:
import torch
# import pickle
from torch.utils.data import DataLoader
from torch.autograd import Variable
from dataset_SNN import Train_Siamese
from model import Siamese
import time
import math
from sklearn.metrics import confusion_matrix, classification_report, f1_score
import pandas as pd
import statistics
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import geomloss
from sklearn import svm
import xgboost as xgb

import random, os
import torch
import numpy as np
seed = 32
random.seed(seed)
os.environ['PYTHONHASHSEED'] =str(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic =True

def init_weights(m):
    if isinstance(m, torch.nn.Linear):
        torch.nn.init.xavier_uniform_(m.weight)
        m.bias.data.fill_(0.01)


filename = '../leave_one_couple_out_analysis/preprocessed_data_valid_couples_v3.csv'
df = pd.read_csv(filename)
all_couples = [105,117,126,133,177,183,188,708,714,715,207,720,721,723,724,725,727,729,730,731,732,733,735,736,737,739,740,741,742,743,744,745,746,747,748,749,750,751,752,753,755,756,757,758,762,763,765,767,771,772,773,774,775,776,777,779,784,786,787,789,791,793,794,796,799,800,907,908,913,924,945,966]

df = df[df['ID'].isin(all_couples)]


pred_global = []
y = []

for couple in all_couples:

    trainSet = Train_Siamese(filename, couple)
    trainLoader = DataLoader(trainSet, batch_size=16, shuffle=True)

    test_df = df.loc[df['ID'] == couple]
    select_count = int(len(test_df) * 0.4)
    test_df = test_df.iloc[select_count: , :]
    test_df_stressed = test_df.loc[test_df['Stress'] != 0].drop(columns=['Stress', 'Gender', 'ID', 'Race', 'Report']).to_numpy()
    test_df_unstressed = test_df.loc[test_df['Stress'] == 0].drop(columns=['Stress', 'Gender', 'ID', 'Race', 'Report']).to_numpy()

    test_X = np.concatenate((test_df_stressed, test_df_unstressed), axis=0)
    test_y = [1]*len(test_df_stressed) + [0]*len(test_df_unstressed)
    y = y+test_y

    test_size = len(test_X)
    test_X = Variable(torch.tensor(np.array(test_X)))
    train_X = np.array(trainSet.train_X)
    train_y = trainSet.train_y

    loss_fn = torch.nn.BCELoss()

    net = Siamese()
    net.apply(init_weights)
    net.train()

    optimizer = torch.optim.Adam(net.parameters(), lr=0.001)
    optimizer.zero_grad()
    running_loss = 0.0

    for epoch in range(10):
        running_loss = 0.0
        for batch_id, (data1, data2, label) in enumerate(trainLoader):
            data1, data2, label = Variable(data1), Variable(data2), Variable(label)
            optimizer.zero_grad()
            output = net.forward(data1, data2)
            loss_Siamese = loss_fn(output, label)

            test_X_embedding = net.forward_one(test_X)

            loss_Wasserstein = []
            if trainSet.stress_ratio >= 0.5:
                ratios = [0.95, 0.5]
            else:
                ratios = [0.05, 0.5]

            scale_list = [int(len(test_X_embedding)*tmp) for tmp in ratios]
            for i in scale_list:
                random_stressed = list(np.random.choice(trainSet.train_stress_count, i, replace=False))
                random_unstressed = list(np.random.choice(trainSet.train_unstress_count, len(test_X_embedding)-i, replace=False) + trainSet.train_stress_count)
                train_X_embedding = Variable(torch.tensor(np.array(train_X[random_stressed+random_unstressed, :])))
                train_X_embedding = net.forward_one(train_X_embedding)
                loss_Wasserstein.append(geomloss.SamplesLoss(blur=0.15)(test_X_embedding, train_X_embedding))
            
            # loss = loss_Siamese
            loss = loss_Siamese + 0.15*min(loss_Wasserstein)/len(test_X_embedding)

            running_loss += loss.detach()
            loss.backward()
            optimizer.step()

        torch.cuda.empty_cache()

    net.eval()
    train_X = Variable(torch.tensor(train_X))
    train_X = net.forward_one(train_X).detach().numpy()
    test_X = Variable(torch.tensor(np.array(test_X)))
    test_X = net.forward_one(test_X).detach().numpy()

    model = svm.SVC(C=1.29, random_state=32)
    model.fit(train_X, train_y)
    pred = list(model.predict(test_X))
    pred_global = pred_global + pred
    
    print('-'*5, 'couple:', couple, '-'*5)
    print('pred:', pred)
    print('true:', test_y)

print(classification_report(y, pred_global, zero_division=0, digits=4), '\n')


/home/kexin/.local/lib/python3.8/site-packages/pkg_resources/__init__.py:122: PkgResourcesDeprecationWarning: 0.1.36ubuntu1 is an invalid version and will not be supported in a future release
  warnings.warn(
/home/kexin/.local/lib/python3.8/site-packages/pkg_resources/__init__.py:122: PkgResourcesDeprecationWarning: 0.23ubuntu1 is an invalid version and will not be supported in a future release
  warnings.warn(


----- couple: 105 -----
pred: [0, 0, 0, 0, 0, 0, 0, 0]
true: [0, 0, 0, 0, 0, 0, 0, 0]
----- couple: 117 -----
pred: [1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0]
true: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
----- couple: 126 -----
pred: [1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]
true: [1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
----- couple: 133 -----
pred: [0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0]
true: [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
----- couple: 177 -----
pred: [0, 0, 0, 0, 0, 0, 0, 0, 0]
true: [1, 1, 0, 0, 0, 0, 0, 0, 0]
----- couple: 183 -----
pred: [0, 1, 0, 1, 0, 1, 0, 1]
true: [1, 0, 0, 0, 0, 0, 0, 0]
----- couple: 188 -----
pred: [1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0]
true: [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0]
----- couple: 708 -----
pred: [0, 1, 0, 1, 1, 1]
true: [1, 1, 1, 1, 1, 1]
----- couple: 714 -----
pred: [1, 1, 1, 1, 1, 1, 1, 1, 1]
true: [1, 1, 1, 1, 1, 1, 0, 0, 0]
----- couple: 715 -----
pred: [1, 1, 0, 0, 0]
true: [1, 1, 0, 0, 0]
----- co

In [2]:
# Copyright (c) Facebook, Inc. and its affiliates. All Rights Reserved
import numpy as np
import os
import subprocess
import sys
from collections import defaultdict
import PIL
import torch
# import torchvision
from tabulate import tabulate

__all__ = ["collect_env_info"]

def get_env_module():
    var_name = "DETECTRON2_ENV_MODULE"
    return var_name, os.environ.get(var_name, "<not set>")


def collect_torch_env():
    try:
        import torch.__config__

        return torch.__config__.show()
    except ImportError:
        # compatible with older versions of pytorch
        from torch.utils.collect_env import get_pretty_env_info

        return get_pretty_env_info()


def collect_env_info():
    data = []
    data.append(("sys.platform", sys.platform))
    data.append(("Python", sys.version.replace("\n", "")))
    data.append(("Numpy", np.__version__))

    data.append(get_env_module())
    data.append(("PyTorch", torch.__version__))
    data.append(("PyTorch Debug Build", torch.version.debug))
    # try:
    #     data.append(("torchvision", torchvision.__version__))
    # except AttributeError:
    #     data.append(("torchvision", "unknown"))

    has_cuda = torch.cuda.is_available()
    data.append(("CUDA available", has_cuda))
    if has_cuda:
        devices = defaultdict(list)
        for k in range(torch.cuda.device_count()):
            devices[torch.cuda.get_device_name(k)].append(str(k))
        for name, devids in devices.items():
            data.append(("GPU " + ",".join(devids), name))

        from torch.utils.cpp_extension import CUDA_HOME

        data.append(("CUDA_HOME", str(CUDA_HOME)))

        if CUDA_HOME is not None and os.path.isdir(CUDA_HOME):
            try:
                nvcc = os.path.join(CUDA_HOME, "bin", "nvcc")
                nvcc = subprocess.check_output("'{}' -V | tail -n1".format(nvcc), shell=True)
                nvcc = nvcc.decode("utf-8").strip()
            except subprocess.SubprocessError:
                nvcc = "Not Available"
            data.append(("NVCC", nvcc))

        cuda_arch_list = os.environ.get("TORCH_CUDA_ARCH_LIST", None)
        if cuda_arch_list:
            data.append(("TORCH_CUDA_ARCH_LIST", cuda_arch_list))
    data.append(("Pillow", PIL.__version__))

    try:
        import cv2

        data.append(("cv2", cv2.__version__))
    except ImportError:
        pass
    env_str = tabulate(data) + "\n"
    env_str += collect_torch_env()
    return env_str


if __name__ == "__main__":
    print(collect_env_info())

---------------------  ---------------------------------------------------
sys.platform           linux
Python                 3.8.10 (default, Jun 22 2022, 20:18:18) [GCC 9.4.0]
Numpy                  1.22.2
DETECTRON2_ENV_MODULE  <not set>
PyTorch                1.10.2+cu102
PyTorch Debug Build    False
CUDA available         True
GPU 0                  NVIDIA GeForce RTX 2070 SUPER
CUDA_HOME              /usr
NVCC                   Cuda compilation tools, release 10.1, V10.1.243
Pillow                 9.0.1
---------------------  ---------------------------------------------------
PyTorch built with:
  - GCC 7.3
  - C++ Version: 201402
  - Intel(R) Math Kernel Library Version 2020.0.0 Product Build 20191122 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v2.2.3 (Git Hash 7336ca9f055cf1bfa13efb658fe15dc9b41f0740)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - CUDA Runtime 10